In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
  

In [30]:
df = pd.read_csv('../data/data.csv')

df.head()

,DateTime,Temperature,Humidity,Wind Speed,general diffuse flows,diffuse flows,Zone 1 Power Consumption,Zone 2 Power Consumption,Zone 3 Power Consumption
0,1/1/2017 0:00,6.559,73.8,0.083,0.051,0.119,34055.69620,16128.87538,20240.96386
1,1/1/2017 0:10,6.414,74.5,0.083,0.070,0.085,29814.68354,19375.07599,20131.08434
2,1/1/2017 0:20,6.313,74.5,0.080,0.062,0.100,29128.10127,19006.68693,19668.43373
3,1/1/2017 0:30,6.121,75.0,0.083,0.091,0.096,28228.86076,18361.09422,18899.27711
4,1/1/2017 0:40,5.921,75.7,0.081,0.048,0.085,27335.69620,17872.34043,18442.40964


# Exploratory Data Analysis and Pre-Processing

- check statistics of each feature
- check for missing values and duplicates
- check correlation (heatmaps, scatter plots...) between columns (to find redundant and irrelevant features
- check for outliers and handle them
- data normalization (minMax or ZScore)


In [31]:
# check if there are duplicates or null values
print("Number of duplicates: ", df.duplicated().sum())
print(df.isnull().sum())

Number of duplicates:  0
DateTime                     0
Temperature                  0
Humidity                     0
Wind Speed                   0
general diffuse flows        0
diffuse flows                0
Zone 1 Power Consumption     0
Zone 2  Power Consumption    0
Zone 3  Power Consumption    0
dtype: int64


# Feature Engineering and Selection
- try to come up with new features (for example from the time column)
- remove irrelevant features with forward and backward selection (optional, as we need to train a model to achieve it)

## Extracting features from DateTIme

In [32]:
df["DateTime"] = pd.to_datetime(df["DateTime"])
df["Year"] = df["DateTime"].dt.year
df["Month"] = df["DateTime"].dt.month
df["Day"] = df["DateTime"].dt.day
df["Hour"] = df["DateTime"].dt.hour
# is weekend (weekend is saturay and sunday)
df["IsWeekend"] = df["DateTime"].dt.weekday.isin([5,6]).astype(int)
# is night (from 20:00 to 06:00) make sure to include both range from 20 to 23 and 0 to 6
df["IsNight"] = df["DateTime"].dt.hour.isin(range(20,24)).astype(int)
df["IsNight"] = df["IsNight"] | df["DateTime"].dt.hour.isin(range(0,7)).astype(int)

# apply cosine and sine transformation to the hour, month and day
df["HourCos"] = np.cos(2 * np.pi * df["Hour"] / 24)
df["HourSin"] = np.sin(2 * np.pi * df["Hour"] / 24)
df["MonthCos"] = np.cos(2 * np.pi * df["Month"] / 12)
df["MonthSin"] = np.sin(2 * np.pi * df["Month"] / 12)
df["DayCos"] = np.cos(2 * np.pi * df["Day"] / 31)
df["DaySin"] = np.sin(2 * np.pi * df["Day"] / 31)


df.head()

,DateTime,Temperature,Humidity,Wind Speed,general diffuse flows,diffuse flows,Zone 1 Power Consumption,Zone 2 Power Consumption,Zone 3 Power Consumption,Year,...,Day,Hour,IsWeekend,IsNight,HourCos,HourSin,MonthCos,MonthSin,DayCos,DaySin
0,2017-01-01 00:00:00,6.559,73.8,0.083,0.051,0.119,34055.69620,16128.87538,20240.96386,2017,...,1,0,1,1,1.0,0.0,0.866025,0.5,0.97953,0.201299
1,2017-01-01 00:10:00,6.414,74.5,0.083,0.070,0.085,29814.68354,19375.07599,20131.08434,2017,...,1,0,1,1,1.0,0.0,0.866025,0.5,0.97953,0.201299
2,2017-01-01 00:20:00,6.313,74.5,0.080,0.062,0.100,29128.10127,19006.68693,19668.43373,2017,...,1,0,1,1,1.0,0.0,0.866025,0.5,0.97953,0.201299
3,2017-01-01 00:30:00,6.121,75.0,0.083,0.091,0.096,28228.86076,18361.09422,18899.27711,2017,...,1,0,1,1,1.0,0.0,0.866025,0.5,0.97953,0.201299
4,2017-01-01 00:40:00,5.921,75.7,0.081,0.048,0.085,27335.69620,17872.34043,18442.40964,2017,...,1,0,1,1,1.0,0.0,0.866025,0.5,0.97953,0.201299


In [33]:
df.describe()

,Temperature,Humidity,Wind Speed,general diffuse flows,diffuse flows,Zone 1 Power Consumption,Zone 2 Power Consumption,Zone 3 Power Consumption,Year,Month,Day,Hour,IsWeekend,IsNight,HourCos,HourSin,MonthCos,MonthSin,DayCos,DaySin
count,52416.000000,52416.000000,52416.000000,52416.000000,52416.000000,52416.000000,52416.000000,52416.000000,52416.0,52416.000000,52416.000000,52416.000000,52416.000000,52416.000000,5.241600e+04,5.241600e+04,5.241600e+04,5.241600e+04,52416.000000,5.241600e+04
mean,18.810024,68.259518,1.959489,182.696614,75.028022,32344.970564,21042.509082,17835.406218,2017.0,6.510989,15.678571,11.500000,0.285714,0.458333,-6.601692e-17,-2.290936e-17,-5.126443e-03,-5.395884e-03,-0.021699,1.636413e-03
std,5.815476,15.551177,2.348862,264.400960,124.210949,7130.562564,5201.465892,6622.165099,0.0,3.440642,8.771822,6.922253,0.451758,0.498266,7.071135e-01,7.071135e-01,7.075804e-01,7.066071e-01,0.699348,7.144631e-01
min,3.247000,11.340000,0.050000,0.004000,0.011000,13895.696200,8560.081466,5935.174070,2017.0,1.000000,1.000000,0.000000,0.000000,0.000000,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-0.994869,-9.987165e-01
25%,14.410000,58.310000,0.078000,0.062000,0.122000,26310.668692,16980.766032,13129.326630,2017.0,4.000000,8.000000,5.750000,0.000000,0.000000,-7.071068e-01,-7.071068e-01,-8.660254e-01,-8.660254e-01,-0.758758,-7.247928e-01
50%,18.780000,69.860000,0.086000,5.035500,4.456000,32265.920340,20823.168405,16415.117470,2017.0,7.000000,16.000000,11.500000,0.000000,0.000000,-6.123234e-17,6.123234e-17,-1.836970e-16,-2.449294e-16,-0.050649,-2.449294e-16
75%,22.890000,81.400000,4.915000,319.600000,101.000000,37309.018185,24713.717520,21624.100420,2017.0,9.250000,23.000000,17.250000,1.000000,1.000000,7.071068e-01,7.071068e-01,5.915064e-01,5.000000e-01,0.688967,7.247928e-01
max,40.010000,94.800000,6.483000,1163.000000,936.000000,52204.395120,37408.860760,47598.326360,2017.0,12.000000,31.000000,23.000000,1.000000,1.000000,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000,9.987165e-01


# Export resulting data after all transformations and pre-processing

In [35]:
# renaming target columns to make them easy to work with in the modeling phase
df.rename(columns={
    "Zone 1 Power Consumption": "z1",
    "Zone 2  Power Consumption": "z2",
    "Zone 3  Power Consumption": "z3"
}, inplace=True)

df.to_csv("data/data-ready.csv", index=False)